### TEST for unitility
Various functions to process the initial bed data

In [82]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from motif_utils import seq2kmer

In [86]:
# file name reader
# make a list of all the filename

bed_dir=os.listdir('../database/bed/unzipped/')
bed_dir

['E001_15_coreMarks_stateno.bed',
 'E002_15_coreMarks_stateno.bed',
 'E003_15_coreMarks_stateno.bed',
 'E004_15_coreMarks_stateno.bed',
 'E005_15_coreMarks_stateno.bed',
 'E006_15_coreMarks_stateno.bed',
 'E007_15_coreMarks_stateno.bed',
 'E008_15_coreMarks_stateno.bed',
 'E009_15_coreMarks_stateno.bed',
 'E010_15_coreMarks_stateno.bed',
 'E011_15_coreMarks_stateno.bed',
 'E012_15_coreMarks_stateno.bed',
 'E013_15_coreMarks_stateno.bed',
 'E014_15_coreMarks_stateno.bed',
 'E015_15_coreMarks_stateno.bed',
 'E016_15_coreMarks_stateno.bed',
 'E017_15_coreMarks_stateno.bed',
 'E018_15_coreMarks_stateno.bed',
 'E019_15_coreMarks_stateno.bed',
 'E020_15_coreMarks_stateno.bed',
 'E021_15_coreMarks_stateno.bed',
 'E022_15_coreMarks_stateno.bed',
 'E023_15_coreMarks_stateno.bed',
 'E024_15_coreMarks_stateno.bed',
 'E025_15_coreMarks_stateno.bed',
 'E026_15_coreMarks_stateno.bed',
 'E027_15_coreMarks_stateno.bed',
 'E028_15_coreMarks_stateno.bed',
 'E029_15_coreMarks_stateno.bed',
 'E030_15_core

In [84]:
def filelist_maker(path):
    filelist=[]
    for file in path:
        filelist.append(file)
    return filelist

In [85]:
filelist_maker(bed_dir)

['.',
 '.',
 '/',
 'd',
 'a',
 't',
 'a',
 'b',
 'a',
 's',
 'e',
 '/',
 'b',
 'e',
 'd',
 '/',
 'u',
 'n',
 'z',
 'i',
 'p',
 'p',
 'e',
 'd']

In [4]:
filename='../database/bed/unzipped/E017_15_coreMarks_stateno.bed'

In [47]:
state_dict={1:"A", 2:"B", 3:"C", 4:"D", 5:"E",6:"F",7:"G",8:"H" ,
                9:"I" ,10:"J",11:"K", 12:"L", 13:"M", 14:"N", 15:"O"}

In [52]:
# create dataframe from bed file
# bed file here means: EXXX_15_coreMarks_stateno.bed

def bed2df_as_is(filename):    
    
    """Create dataframe from the .bed file, as is.
    Dataframe contains following columns:
    chromosome |  start |  end  | state """
    
    df_raw=pd.read_csv(filename, sep='\t', lineterminator='\n', header=None, low_memory=False)
    df=df_raw.rename(columns={0:"chromosome",1:"start",2:"end",3:"state"})
    df=df[:-1]
    df["start"]=pd.to_numeric(df["start"])
    df["end"]=pd.to_numeric(df["end"])
    
    return df

In [53]:
def bed2df_expanded(filename):
    
    """Create an expanded dataframe from the .bed file.
    Dataframe contains following columns:
    chromosome |  start |  end  | state | length | unit | state_seq | state_seq_full"""
   
    df_raw=pd.read_csv(filename, sep='\t', lineterminator='\n', header=None, low_memory=False)
    df=df_raw.rename(columns={0:"chromosome",1:"start",2:"end",3:"state"})
    df=df[:-1]
    df["start"]=pd.to_numeric(df["start"])
    df["end"]=pd.to_numeric(df["end"])
    df["state"]=pd.to_numeric(df["state"])
    df["length"]=df["end"]-df["start"]
    df["unit"]=(df["length"]/100).astype(int)
               
    df["state_seq"]=df["state"].map(state_dict)
    df["state_seq_full"]=df["unit"]*df["state_seq"]
    
    return df 

In [78]:
df_asis=bed2df_as_is(filename)
df_asis.head()

,chromosome,start,end,state
0,chr1,0,9800,15
1,chr1,9800,10800,9
2,chr1,10800,13000,15
3,chr1,13000,13200,7
4,chr1,13200,16000,5


In [59]:
df=bed2df_expanded(filename)
df.head()

,chromosome,start,end,state,length,unit,state_seq,state_seq_full
0,chr1,0,9800,15,9800,98,O,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1,chr1,9800,10800,9,1000,10,I,IIIIIIIIII
2,chr1,10800,13000,15,2200,22,O,OOOOOOOOOOOOOOOOOOOOOO
3,chr1,13000,13200,7,200,2,G,GG
4,chr1,13200,16000,5,2800,28,E,EEEEEEEEEEEEEEEEEEEEEEEEEEEE


In [79]:
def numchr(df):
    assert "chromosome" in df.columns, "Check your df has the column named 'chromosome'"
    return df["chromosome"].nunique()    

In [67]:
numchr(df)

25

In [ ]:
# To collect the number of sequences for each chromosome, scan the start column
total_row=len(df)
chr_len=[]
chr_check=[]
chr_index=[]

for i in range(total_row):
    if (df["start"].iloc[i]==0) & (i >0):
        chr_len.append(df["end"].iloc[i-1])
        chr_check.append(df["start"].iloc[i]) # assert
        chr_index.append(i-1)


end_len=df["end"].iloc[-1]
end_index=total_row-1

chr_len.append(end_len)
chr_index.append(end_index)

assert len(chr_len)==df["chromosome"].nunique()
assert len(chr_index)==df["chromosome"].nunique()

In [74]:
class bed2df_cls:
    
    def __init__(self, fname):
        
        self.fname=fname
        
        df_raw=pd.read_csv(fname, sep='\t', lineterminator='\n', 
                           header=None, low_memory=False)
        df=df_raw.rename(columns={0:"chromosome",1:"start",2:"end",3:"state"})
        df=df[:-1] # remove the end row: it displayed the cell id and track no.
        df["start"]=pd.to_numeric(df["start"])
        df["end"]=pd.to_numeric(df["end"])
        df["length"]=df["end"]-df["start"]
        df["unit"]=(df["length"]/100).astype(int)
        
        state_dict={1:"A", 2:"B", 3:"C", 4:"D", 5:"E",6:"F",7:"G",8:"H" ,
                    9:"I" ,10:"J",11:"K", 12:"L", 13:"M", 14:"N", 15:"O"}
        
        df["state"]=pd.to_numeric(df["state"])
        df["state_seq"]=df["state"].map(state_dict)
        df["state_seq_full"]=df["unit"]*df["state_seq"]
        
        self.df=df
        self.df_len=len(df)
        self.numchr=df["chromosome"].nunique()
        
        print(".df : dataframe \n.df_len : length of dataframe \n.numchr : no. of chromosome")
        
     

In [76]:
df_test=bed2df_cls(filename)
df_test.df.head()

.df : dataframe 
.df_len : length of dataframe 
.numchr : no. of chromosome


,chromosome,start,end,state,length,unit,state_seq,state_seq_full
0,chr1,0,9800,15,9800,98,O,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1,chr1,9800,10800,9,1000,10,I,IIIIIIIIII
2,chr1,10800,13000,15,2200,22,O,OOOOOOOOOOOOOOOOOOOOOO
3,chr1,13000,13200,7,200,2,G,GG
4,chr1,13200,16000,5,2800,28,E,EEEEEEEEEEEEEEEEEEEEEEEEEEEE
